---
# Imports

In [1]:
import pyspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import numpy as np
#!pip install kagglehub
#import kagglehub

In [2]:
spark = SparkSession.builder.appName("ProjetoABD").getOrCreate()

In [16]:
#dados = spark.read.parquet('avioes/pq').limit(50000)

#dados = spark.read.load('avioes/csv',format='csv',sep=',',inferSchema=True, header=True)#.limit(50000)

#Import Bruno
dados = spark.read.load('/home/jovyan/code/raw',format='csv',sep=',',inferSchema=True, header=True)#.limit(50000)




In [17]:
dados.printSchema()
dados.show()
dados.count()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: date (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT_ID_Marketing_Airline: integer (nullable = true)
 |-- IATA_Code_Marketing_Airline: string (nullable = true)
 |-- Flight_Number_Marketing_Airline: integer (nullable = true)
 |-- Originally_Scheduled_Code_Share_Airline: string (nullable = true)
 |-- DOT_ID_Originally_Scheduled_Code_Share_Airline: integer (nullable = true)
 |-- IATA_Code_Originally_Scheduled_Code_Share_Airline: string (nullable = true)
 |-- Flight_Num_Originally_Scheduled_Code_Share_Airline: integer (nullable = true)
 |-- Operating_Airline : string (nullable = true)
 |-- DOT_ID_Operating_Airline: integer (nullable = true)
 |-- IATA_Code_Operating_A

29193782

-----
# Data Cleaning

## 1.Removing duplicates if any

In [10]:
# Checking first: use dropDuplicates() and count()
print(f'Dados: number of rows: {dados.count()}, after dropduplicates: {dados.dropDuplicates().count() }')

Dados: number of rows: 29193782, after dropduplicates: 29193782


## 2.Removing useless columns

In [ ]:
# apagar o Year|Quarter|Month|DayofMonth -
# substituir os OriginCityName|OriginState|OriginStateName|OriginStateFips num só
# DestCityName|DestState|DestStateNam|DestStateFips mesmo que em cima
# tirar o OriginCityMarketID que ja temos a cidade -
# tirar OriginAirportID|OriginAirportSeqID que ja temos o nome do aeroporto -
# igual para estes DestAirportID|DestAirportSeqID|DestCityMarketID -
# Marketing_Airline_Network IATA_Code_Marketing_Airline iguais
# Operating_Airline IATA_Code_Operating_Airline iguais
# Flight_Number_Marketing_Airline Flight_Number_Operating_Airline iguais
# apagar DepDelayMinutes e DepDel15 e arranjar o DepDelay ! fazer o mesmo para ArrDelay ArrDelayMinutes ArrDel15

In [18]:
cols_to_dismiss = ['DestAirportID','DestAirportSeqID','DestCityMarketID','OriginAirportID','OriginAirportSeqID'
                   ,'OriginCityMarketID','Year','Quarter','Month','DayofMonth','Marketing_Airline_Network'
                   ,'Operating_Airline','Flight_Number_Marketing_Airline','ArrDelayMinutes','DepDelayMinutes']
dados = dados.drop(*cols_to_dismiss)

In [19]:
dados = dados.withColumn('OriginCityNameState',
                          F.concat_ws(
                              ',',
                                F.split(F.col('OriginCityName'),',')[0],
                                F.col("OriginStateName")
    ))
#dados.show()

In [20]:
dados = dados.withColumn('DestCityNameState',
                          F.concat_ws(
                              ',',
                                F.split(F.col('DestCityName'),',')[0],
                                F.col("DestStateName")
    ))
#dados.show()

In [21]:
cols_to_dismiss = ["OriginCityName","OriginState","OriginStateName","OriginStateFips","DestCityName","DestState","DestStateNam","DestStateFips"]
dados = dados.drop(*cols_to_dismiss)

## 3.Handling missing values

In [ ]:
# dados.groupby("DivAirportLandings").count().show()

# apagar DepDelayMinutes e DepDel15 e arranjar o DepDelay ! fazer o mesmo para ArrDelay ArrDelayMinutes ArrDel15

In [29]:
#dados.show()
# 'DepTime': 761652
#dados = dados.withColumn('DepDelay', F.col('DepTime')-F.col('CRSDepTime'))


In [ ]:
# Ver em que colunas estão os nulos (by columns)
dict_nulls_calls = {col: dados.filter(dados[col].isNull()).count() for col in dados.columns}
dict_nulls_calls

# valores corridos ao dataset todo sem fazer nada aos nulls

{'FlightDate': 0,
 'Airline': 0,
 'Origin': 0,
 'Dest': 0,
 'Cancelled': 0,
 'Diverted': 0,
 'CRSDepTime': 0,
 'DepTime': 761652,
 'DepDelayMinutes': 763084,
 'DepDelay': 763084,
 'ArrTime': 786177,
 'ArrDelayMinutes': 846183,
 'AirTime': 852561,
 'CRSElapsedTime': 22,
 'ActualElapsedTime': 845637,
 'Distance': 0,
 'DayOfWeek': 0,
 'Operated_or_Branded_Code_Share_Partners': 0,
 'DOT_ID_Marketing_Airline': 0,
 'IATA_Code_Marketing_Airline': 0,
 'DOT_ID_Operating_Airline': 0,
 'IATA_Code_Operating_Airline': 0,
 'Tail_Number': 267613,
 'Flight_Number_Operating_Airline': 0,
 'OriginWac': 0,
 'DestStateName': 0,
 'DestWac': 0,
 'DepDel15': 763084,
 'DepartureDelayGroups': 763084,
 'DepTimeBlk': 0,
 'TaxiOut': 780561,
 'WheelsOff': 780551,
 'WheelsOn': 793133,
 'TaxiIn': 793143,
 'CRSArrTime': 0,
 'ArrDelay': 846183,
 'ArrDel15': 846183,
 'ArrivalDelayGroups': 846183,
 'ArrTimeBlk': 0,
 'DistanceGroup': 0,
 'DivAirportLandings': 90,
 'OriginCityNameState': 0,
 'DestCityNameState': 0}

In [32]:
#Quantas linhas ficavam depois tem valores nulos
dados.dropna(how='any').count()

5005391

In [ ]:


cols_to_dismiss = [
    # Informações duplicadas ou derivadas
    'DepDelayMinutes', 'DepDelay', 'DepDel15', 'DepartureDelayGroups',
    'ArrDelayMinutes', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups',
    'AirTime', 'ActualElapsedTime', 'WheelsOff', 'WheelsOn', 'Tail_Number',

    # Diverted flights – detalhes específicos desnecessários
    'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime',
    'DivArrDelay', 'DivDistance',

    # Todas as colunas relacionadas com aeroportos desviados 
    'Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn',
    'Div1TotalGTime', 'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum',
    'Div2Airport', 'Div2AirportID', 'Div2AirportSeqID', 'Div2WheelsOn',
    'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum',
    'Div3Airport', 'Div3AirportID', 'Div3AirportSeqID', 'Div3WheelsOn',
    'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 'Div3TailNum',
    'Div4Airport', 'Div4AirportID', 'Div4AirportSeqID', 'Div4WheelsOn',
    'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 'Div4TailNum',
    'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn',
    'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum',

    # Identificadores redundantes
    'DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline',
    'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline',

    'CancellationCode'
]

dados = dados.drop(*cols_to_dismiss)

In [ ]:
dict_nulls_calls = {col: dados.filter(dados[col].isNull()).count() for col in dados.columns}
dict_nulls_calls

{'DayOfWeek': 0,
 'FlightDate': 0,
 'Operated_or_Branded_Code_Share_Partners': 0,
 'Originally_Scheduled_Code_Share_Airline': 29189909,
 'DOT_ID_Originally_Scheduled_Code_Share_Airline': 29189909,
 'IATA_Code_Originally_Scheduled_Code_Share_Airline': 29189909,
 'Flight_Num_Originally_Scheduled_Code_Share_Airline': 29189909,
 'Operating_Airline ': 0,
 'Flight_Number_Operating_Airline': 0,
 'Origin': 0,
 'OriginWac': 0,
 'Dest': 0,
 'DestStateName': 0,
 'DestWac': 0,
 'CRSDepTime': 0,
 'DepTime': 761652,
 'DepTimeBlk': 0,
 'TaxiOut': 780561,
 'TaxiIn': 793143,
 'CRSArrTime': 0,
 'ArrTime': 786177,
 'ArrTimeBlk': 0,
 'Cancelled': 0,
 'Diverted': 0,
 'CRSElapsedTime': 22,
 'Flights': 0,
 'Distance': 0,
 'DistanceGroup': 0,
 'CarrierDelay': 24187011,
 'WeatherDelay': 24187014,
 'NASDelay': 24187014,
 'SecurityDelay': 24187014,
 'LateAircraftDelay': 24187014,
 'FirstDepTime': 29010646,
 'TotalAddGTime': 29010666,
 'LongestAddGTime': 29010665,
 'Duplicate': 0,
 '_c119': 29193782,
 'OriginCityNameState': 0,
 'DestCityNameState': 0}

In [38]:


cols_to_dismiss = [
    'Originally_Scheduled_Code_Share_Airline','DOT_ID_Originally_Scheduled_Code_Share_Airline','IATA_Code_Originally_Scheduled_Code_Share_Airline',
    'Flight_Num_Originally_Scheduled_Code_Share_Airline','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','FirstDepTime','TotalAddGTime',
    'LongestAddGTime','_c119','LateAircraftDelay'
]

dados = dados.drop(*cols_to_dismiss)

In [ ]:
dict_nulls_calls = {col: dados.filter(dados[col].isNull()).count() for col in dados.columns}
dict_nulls_calls

{'DayOfWeek': 0,
 'FlightDate': 0,
 'Operated_or_Branded_Code_Share_Partners': 0,
 'Operating_Airline ': 0,
 'Flight_Number_Operating_Airline': 0,
 'Origin': 0,
 'OriginWac': 0,
 'Dest': 0,
 'DestStateName': 0,
 'DestWac': 0,
 'CRSDepTime': 0,
 'DepTime': 761652,
 'DepTimeBlk': 0,
 'TaxiOut': 780561,
 'TaxiIn': 793143,
 'CRSArrTime': 0,
 'ArrTime': 786177,
 'ArrTimeBlk': 0,
 'Cancelled': 0,
 'Diverted': 0,
 'CRSElapsedTime': 22,
 'Flights': 0,
 'Distance': 0,
 'DistanceGroup': 0,
 'Duplicate': 0,
 'OriginCityNameState': 0,
 'DestCityNameState': 0}

In [40]:
#Quantas linhas ficavam depois tem valores nulos
dados.dropna(how='any').count()

28400626

-----
# Data Transformation

In [ ]:
# converter hhmm para minutos desde meia-noite
def hhmm_to_minutes(col):
    return (F.floor(col / 100) * 60) + (col % 100)

#  tempos de partida
dados = dados.withColumn("DepTime_min", hhmm_to_minutes(F.col("DepTime")))
dados = dados.withColumn("CRSDepTime_min", hhmm_to_minutes(F.col("CRSDepTime")))
dados = dados.withColumn("DepDelay", F.col("DepTime_min") - F.col("CRSDepTime_min"))

#  tempos de chegada
dados = dados.withColumn("ArrTime_min", hhmm_to_minutes(F.col("ArrTime")))
dados = dados.withColumn("CRSArrTime_min", hhmm_to_minutes(F.col("CRSArrTime")))
dados = dados.withColumn("ArrDelay", F.col("ArrTime_min") - F.col("CRSArrTime_min"))



In [44]:
dados.show(1)

+---------+----------+---------------------------------------+------------------+-------------------------------+------+---------+----+-------------+-------+----------+-------+----------+-------+------+----------+-------+----------+---------+--------+--------------+-------+--------+-------------+---------+--------------------+-------------------+--------+-----------+--------------+-----------+--------------+--------+
|DayOfWeek|FlightDate|Operated_or_Branded_Code_Share_Partners|Operating_Airline |Flight_Number_Operating_Airline|Origin|OriginWac|Dest|DestStateName|DestWac|CRSDepTime|DepTime|DepTimeBlk|TaxiOut|TaxiIn|CRSArrTime|ArrTime|ArrTimeBlk|Cancelled|Diverted|CRSElapsedTime|Flights|Distance|DistanceGroup|Duplicate| OriginCityNameState|  DestCityNameState|DepDelay|DepTime_min|CRSDepTime_min|ArrTime_min|CRSArrTime_min|ArrDelay|
+---------+----------+---------------------------------------+------------------+-------------------------------+------+---------+----+-------------+-------+-